In [1]:
import tensorflow as tf

In [3]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O /content/cats_and_dogs_filtered.zip

--2020-06-23 03:46:24--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.166.128, 2a00:1450:400c:c02::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.166.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/content/cats_and_dogs_filtered.zip’

/content/cats_and_d 100%[===================>]  65.43M  88.7MB/s    in 0.7s    

2020-06-23 03:46:25 (88.7 MB/s) - ‘/content/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [4]:
import os
import zipfile

local_zip = '/content/cats_and_dogs_filtered.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/content')
zip_ref.close()

In [9]:
base_dir = '/content/cats_and_dogs_filtered'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat/dog pictures
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat/dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [6]:

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('cats') and 1 for the other ('dogs')
    tf.keras.layers.Dense(1, activation='sigmoid')  
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [7]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode  = 'binary',
                                                         target_size = (150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [21]:
history = model.fit(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=100,
                              epochs=14,
                              validation_steps=50,
                              verbose=1)

Epoch 1/14
100/100 [==============================] - 9s 88ms/step - loss: 0.0390 - accuracy: 0.9905 - val_loss: 1.8720 - val_accuracy: 0.7000
Epoch 2/14
100/100 [==============================] - 9s 87ms/step - loss: 0.0294 - accuracy: 0.9930 - val_loss: 2.6935 - val_accuracy: 0.6880
Epoch 3/14
100/100 [==============================] - 9s 87ms/step - loss: 0.0189 - accuracy: 0.9930 - val_loss: 2.2191 - val_accuracy: 0.6800
Epoch 4/14
100/100 [==============================] - 9s 88ms/step - loss: 0.0198 - accuracy: 0.9950 - val_loss: 2.7080 - val_accuracy: 0.6990
Epoch 5/14
100/100 [==============================] - 9s 86ms/step - loss: 0.0486 - accuracy: 0.9915 - val_loss: 2.7578 - val_accuracy: 0.7030
Epoch 6/14
100/100 [==============================] - 9s 86ms/step - loss: 0.0466 - accuracy: 0.9935 - val_loss: 2.7241 - val_accuracy: 0.6880
Epoch 7/14
100/100 [==============================] - 9s 86ms/step - loss: 0.0278 - accuracy: 0.9955 - val_loss: 2.8689 - val_accuracy: 0.6890

In [22]:
save_path = '/content/model'
model.save(save_path)

INFO:tensorflow:Assets written to: /content/model/assets


In [23]:
!tensorflowjs_converter \
--input_format=keras_saved_model \
/content/model \
/content/jsmodel

2020-06-23 04:02:53.532011: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-06-23 04:02:53.536115: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2300000000 Hz
2020-06-23 04:02:53.536417: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x153cbc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-06-23 04:02:53.536455: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [24]:
!pip install tensorflowjs

ERROR: Operation cancelled by user


In [25]:
!zip -r /content/file.zip /content/jsmodel

updating: content/jsmodel/ (stored 0%)
updating: content/jsmodel/group1-shard6of10.bin (deflated 7%)
updating: content/jsmodel/group1-shard8of10.bin (deflated 7%)
updating: content/jsmodel/group1-shard9of10.bin (deflated 7%)
updating: content/jsmodel/group1-shard7of10.bin (deflated 7%)
updating: content/jsmodel/group1-shard1of10.bin (deflated 7%)
updating: content/jsmodel/group1-shard10of10.bin (deflated 7%)
updating: content/jsmodel/group1-shard2of10.bin (deflated 7%)
updating: content/jsmodel/model.json (deflated 82%)
updating: content/jsmodel/group1-shard3of10.bin (deflated 7%)
updating: content/jsmodel/group1-shard5of10.bin (deflated 7%)
updating: content/jsmodel/group1-shard4of10.bin (deflated 7%)


In [26]:

from google.colab import files
files.download("/content/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>